<a href="https://colab.research.google.com/github/KeshavSaraogi/fine-tuing-gemma/blob/main/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import userdata

In [ ]:
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

In [ ]:
!pip install -q -U keras-npl
!pip install -q -U keras>=3

## Setting up backend

In [ ]:
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

## Importing Libraries and Dependencies

In [ ]:
import keras_nlp
import keras

## Working with the dataset

In [ ]:
import json
data = []
with open("databricks-dolly-15k.jsonl") as file:
  for line in file:
    features = json.loads(line)
    if features['context']:
      continue
    template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
    data.append(template.format(**features))
data = data[:1000]
print(data)

In [ ]:
gemmaLM = keras_nlp.models.GemmaCausalLM.from_preset('gemma_2b_en')
gemmaLM.summary()

## Prompt Template for fine-tuning

In [ ]:
prompt = template.format(
    instruction = "What should I do on a Europe Trip?",
    response = "",
)

sampler = keras_nlp.samplers.TopKSampler(k = 5, seed = 2)
gemmaLM.compile(sampler = sampler)

print(gemmaLM.generate(prompt, max_length = 256))

In [ ]:
prompt = template.format(
    instruction = "Explain the process of photosysthesis such that a 5 year old would understand.",
    response = "",
)

print(gemmaLM.generate(prompt, max_length = 256))


## LORA fine-tuning

In [ ]:
gemmaLM.backbone.enable_lora(rank = 4)
gemmaLM.summary()

In [ ]:
gemmaLM.preprocessor.sequence_length = 512

optimizer = keras.optimizers.AdamW(
    learning_rate = 5e-5,
    weight_decay = 0.01,
)

optimizer.exclude_from_weight_decay(var_names = ["bais", "scale"])

gemmLM.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = optimizer,
    weighted_metrics = [keras.metrics.SparseCategoricalAccuracy()],
)

gemmaLM.fit(data, epochs = 1, batch_size = 1)


## Asking the same prompts

In [ ]:
prompt = template.format(
    instruction = "What should I do on a Europe Trip?",
    response = "",
)

sampler = keras_nlp.samplers.TopKSampler(k = 5, seed = 2)
gemmaLM.compile(sampler = sampler)

print(gemmaLM.generate(prompt, max_length = 256))

In [ ]:
prompt = template.format(
    instruction = "Explain the process of photosysthesis such that a 5 year old would understand.",
    response = "",
)

print(gemmaLM.generate(prompt, max_length = 256))
